In [17]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [18]:
df=pd.read_csv('/home/gaurav/Documents/Mor_pred/ayush/train.csv')
labels=pd.read_csv('/home/gaurav/Documents/Mor_pred/ayush/labels_train.csv')

In [19]:
df.head()


,ALP,ALT,AST,Age,Albumin,BUN,Bilirubin,Cholesterol,Creatinine,DiasABP,...,RespRate,SaO2,SysABP,Temp,TroponinI,TroponinT,Urine,WBC,Weight,pH
0,77.0,31.0,46.0,54,2.973333,10.500000,0.7,154.0,0.750000,58.795833,...,17.428571,97.250000,116.891892,37.357143,2.1,0.14,171.052632,10.300000,80.060976,7.387273
1,77.0,31.0,46.0,76,2.973333,18.333333,0.7,154.0,1.100000,58.897059,...,19.000000,96.833333,113.411765,36.939130,2.1,0.14,151.560976,11.266667,80.670588,7.395000
2,116.0,83.0,199.5,44,2.500000,4.666667,2.9,154.0,0.333333,67.125000,...,19.000000,95.000000,125.687500,37.800000,2.1,0.14,124.951220,4.700000,56.700000,7.495000
3,105.0,12.0,15.0,68,4.400000,17.666667,0.2,154.0,0.766667,58.795833,...,15.457627,97.250000,116.891892,36.223077,2.1,0.14,545.833333,9.400000,84.600000,7.387273
4,77.0,31.0,46.0,88,3.300000,35.000000,0.7,154.0,1.000000,58.795833,...,19.166667,97.250000,116.891892,36.880000,2.1,0.14,62.131579,4.300000,80.060976,7.387273


In [20]:
from sklearn.preprocessing import MinMaxScaler
#normalization
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)


In [21]:
from imblearn.over_sampling import SMOTE

df_res,labels_res=SMOTE(random_state=9).fit_sample(df,labels) #balancing the data
df_res=pd.DataFrame(df_res)
labels_res=pd.DataFrame(labels_res)



/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(sparse=False)
onehot_encoded = enc.fit_transform(labels_res)

labels_res=onehot_encoded

In [23]:
'''corrmat=df.corr()
f, ax = plt.subplots(figsize=(12, 12))
sns.heatmap(corrmat,vmax=.6,square=True)
plt.show()'''

from sklearn.model_selection import train_test_split

train_data,test_data,train_labels,test_labels=train_test_split(df_res,labels_res,random_state=3,train_size=0.8)

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [24]:
import tensorflow as tf
sess=tf.InteractiveSession()

In [25]:
#batch_xs, batch_ys = tf.train.batch([train_data, train_labels],batch_size=100)

n_hidden_1=60
n_hidden_2=60
n_hidden_3=60
n_hidden_4=60

x=tf.placeholder(tf.float32,shape=[None,42])
y_= tf.placeholder(tf.float32, shape=[None, 2])


w=tf.Variable(tf.zeros([42,2]))
b=tf.Variable(tf.zeros([2]))

def multilayer_perceptron(x,weights,biases):
    
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.sigmoid(layer_3)
    
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.relu(layer_4)
    
    
    out_layer=tf.add(tf.matmul(layer_4,weights['out']),biases['out'])
    return out_layer

weights={
            'h1':tf.Variable(tf.truncated_normal([42,n_hidden_1])),
            'h2':tf.Variable(tf.truncated_normal([n_hidden_1,n_hidden_2])),
            'h3':tf.Variable(tf.truncated_normal([n_hidden_2,n_hidden_3])),
            'h4':tf.Variable(tf.truncated_normal([n_hidden_3,n_hidden_4])),
            'out':tf.Variable(tf.truncated_normal([n_hidden_4,2]))
}

biases={
            'b1':tf.Variable(tf.truncated_normal([n_hidden_1])),
            'b2':tf.Variable(tf.truncated_normal([n_hidden_2])),
            'b3':tf.Variable(tf.truncated_normal([n_hidden_3])),
            'b4':tf.Variable(tf.truncated_normal([n_hidden_4])),
            'out':tf.Variable(tf.truncated_normal([2]))
}

y=multilayer_perceptron(x,weights,biases)
cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y))
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

sess.run(tf.global_variables_initializer())

In [26]:
mse_history=[]
cost_history=[]
accuracy_history=[]

In [27]:
training_epochs=1000

for epoch in range(training_epochs):
    sess.run(train_step,feed_dict={x:train_data,y_:train_labels})
    cost=sess.run(cross_entropy,feed_dict={x:train_data,y_:train_labels})
    cost_history=np.append(cost_history,cost)
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    pred_y=sess.run(y,feed_dict={x:test_data})
    mse=tf.reduce_mean(tf.square(pred_y - test_labels))
    mse_=sess.run(mse)
    mse_history.append(mse_)
    accuracy=sess.run(accuracy,feed_dict={x:train_data,y_:train_labels})
    accuracy_history.append(accuracy)
    
    print('epoch:',epoch,'-','cost:',cost,'-MSE:',mse_,'-train accuracy:',accuracy)

epoch: 0 - cost: 10.1045 -MSE: 209.595620367 -train accuracy: 0.50127
epoch: 1 - cost: 1.56659 -MSE: 117.753665234 -train accuracy: 0.521045
epoch: 2 - cost: 1.50624 -MSE: 116.837039258 -train accuracy: 0.532293
epoch: 3 - cost: 1.51259 -MSE: 117.838177217 -train accuracy: 0.51016
epoch: 4 - cost: 2.02456 -MSE: 116.024502442 -train accuracy: 0.511792
epoch: 5 - cost: 4.6514 -MSE: 144.27980891 -train accuracy: 0.499093
epoch: 6 - cost: 4.38745 -MSE: 124.949171315 -train accuracy: 0.500907
epoch: 7 - cost: 4.65309 -MSE: 139.691511314 -train accuracy: 0.499274
epoch: 8 - cost: 4.14449 -MSE: 117.429488311 -train accuracy: 0.500907
epoch: 9 - cost: 4.6097 -MSE: 134.14253989 -train accuracy: 0.499274
epoch: 10 - cost: 3.93651 -MSE: 108.517912548 -train accuracy: 0.500907
epoch: 11 - cost: 4.50546 -MSE: 127.533649534 -train accuracy: 0.499274
epoch: 12 - cost: 3.80035 -MSE: 99.0731934463 -train accuracy: 0.501089
epoch: 13 - cost: 4.35027 -MSE: 122.466773522 -train accuracy: 0.499274
epoch: 1

epoch: 115 - cost: 2.10923 -MSE: 63.8499513806 -train accuracy: 0.499274
epoch: 116 - cost: 2.32434 -MSE: 56.7372436834 -train accuracy: 0.501451
epoch: 117 - cost: 2.09523 -MSE: 63.1295729545 -train accuracy: 0.499274
epoch: 118 - cost: 2.30265 -MSE: 56.1083473452 -train accuracy: 0.501451
epoch: 119 - cost: 2.08132 -MSE: 62.4213654258 -train accuracy: 0.499274
epoch: 120 - cost: 2.28136 -MSE: 55.4935070613 -train accuracy: 0.501451
epoch: 121 - cost: 2.06759 -MSE: 61.7233786951 -train accuracy: 0.499274
epoch: 122 - cost: 2.2605 -MSE: 54.8908386643 -train accuracy: 0.501451
epoch: 123 - cost: 2.05417 -MSE: 61.0425435146 -train accuracy: 0.499274
epoch: 124 - cost: 2.24019 -MSE: 54.300125189 -train accuracy: 0.50127
epoch: 125 - cost: 2.04078 -MSE: 60.3689089939 -train accuracy: 0.499274
epoch: 126 - cost: 2.22034 -MSE: 53.711980537 -train accuracy: 0.50127
epoch: 127 - cost: 2.02752 -MSE: 59.7020890942 -train accuracy: 0.499274
epoch: 128 - cost: 2.20086 -MSE: 53.1321408237 -train ac

epoch: 228 - cost: 1.54697 -MSE: 33.1190658372 -train accuracy: 0.508527
epoch: 229 - cost: 1.49213 -MSE: 36.6349396615 -train accuracy: 0.500544
epoch: 230 - cost: 1.5372 -MSE: 32.8165764152 -train accuracy: 0.50889
epoch: 231 - cost: 1.48362 -MSE: 36.2986432481 -train accuracy: 0.500726
epoch: 232 - cost: 1.52751 -MSE: 32.5164106358 -train accuracy: 0.50889
epoch: 233 - cost: 1.47514 -MSE: 35.9648520649 -train accuracy: 0.500544
epoch: 234 - cost: 1.51798 -MSE: 32.218810364 -train accuracy: 0.509434
epoch: 235 - cost: 1.46669 -MSE: 35.6339881487 -train accuracy: 0.500907
epoch: 236 - cost: 1.50855 -MSE: 31.9239466027 -train accuracy: 0.509615
epoch: 237 - cost: 1.45831 -MSE: 35.3056254848 -train accuracy: 0.500907
epoch: 238 - cost: 1.49914 -MSE: 31.6324047831 -train accuracy: 0.509978
epoch: 239 - cost: 1.45007 -MSE: 34.9812393434 -train accuracy: 0.501089
epoch: 240 - cost: 1.48976 -MSE: 31.3412918531 -train accuracy: 0.509797
epoch: 241 - cost: 1.44188 -MSE: 34.6587089054 -train a

epoch: 341 - cost: 1.09583 -MSE: 21.4692145165 -train accuracy: 0.519049
epoch: 342 - cost: 1.10546 -MSE: 19.3048093912 -train accuracy: 0.525218
epoch: 343 - cost: 1.09035 -MSE: 21.2513514136 -train accuracy: 0.519775
epoch: 344 - cost: 1.09966 -MSE: 19.1091020411 -train accuracy: 0.525399
epoch: 345 - cost: 1.08488 -MSE: 21.0353669121 -train accuracy: 0.519775
epoch: 346 - cost: 1.09393 -MSE: 18.9161030216 -train accuracy: 0.525762
epoch: 347 - cost: 1.07952 -MSE: 20.8214977763 -train accuracy: 0.519775
epoch: 348 - cost: 1.08833 -MSE: 18.725301437 -train accuracy: 0.526306
epoch: 349 - cost: 1.07411 -MSE: 20.6125009995 -train accuracy: 0.520682
epoch: 350 - cost: 1.08264 -MSE: 18.5381324816 -train accuracy: 0.527213
epoch: 351 - cost: 1.06883 -MSE: 20.4067537337 -train accuracy: 0.521589
epoch: 352 - cost: 1.07697 -MSE: 18.3510938766 -train accuracy: 0.527758
epoch: 353 - cost: 1.0635 -MSE: 20.2003287716 -train accuracy: 0.522496
epoch: 354 - cost: 1.0714 -MSE: 18.1651901684 -train 

epoch: 453 - cost: 0.853562 -MSE: 13.0121873092 -train accuracy: 0.565493
epoch: 454 - cost: 0.855395 -MSE: 11.7816116648 -train accuracy: 0.569122
epoch: 455 - cost: 0.850447 -MSE: 12.9179632595 -train accuracy: 0.567126
epoch: 456 - cost: 0.852235 -MSE: 11.6994939931 -train accuracy: 0.570392
epoch: 457 - cost: 0.847328 -MSE: 12.8262744509 -train accuracy: 0.567852
epoch: 458 - cost: 0.84906 -MSE: 11.6193192081 -train accuracy: 0.572025
epoch: 459 - cost: 0.844246 -MSE: 12.7350729315 -train accuracy: 0.568941
epoch: 460 - cost: 0.845937 -MSE: 11.540094258 -train accuracy: 0.571843
epoch: 461 - cost: 0.841236 -MSE: 12.6449582298 -train accuracy: 0.569848
epoch: 462 - cost: 0.842816 -MSE: 11.461113372 -train accuracy: 0.572388
epoch: 463 - cost: 0.838202 -MSE: 12.5560034727 -train accuracy: 0.571299
epoch: 464 - cost: 0.839774 -MSE: 11.3846495459 -train accuracy: 0.573113
epoch: 465 - cost: 0.835273 -MSE: 12.468438959 -train accuracy: 0.571843
epoch: 466 - cost: 0.836781 -MSE: 11.30807

epoch: 565 - cost: 0.693789 -MSE: 8.72354047333 -train accuracy: 0.629717
epoch: 566 - cost: 0.693406 -MSE: 7.97300836507 -train accuracy: 0.627721
epoch: 567 - cost: 0.691397 -MSE: 8.67837397948 -train accuracy: 0.631168
epoch: 568 - cost: 0.691078 -MSE: 7.93513435279 -train accuracy: 0.62881
epoch: 569 - cost: 0.689135 -MSE: 8.6362203189 -train accuracy: 0.633345
epoch: 570 - cost: 0.688858 -MSE: 7.89896790417 -train accuracy: 0.629898
epoch: 571 - cost: 0.686986 -MSE: 8.5953393853 -train accuracy: 0.63516
epoch: 572 - cost: 0.686809 -MSE: 7.86637673789 -train accuracy: 0.630443
epoch: 573 - cost: 0.684996 -MSE: 8.55821145415 -train accuracy: 0.635704
epoch: 574 - cost: 0.684927 -MSE: 7.83732396057 -train accuracy: 0.630805
epoch: 575 - cost: 0.683221 -MSE: 8.52374557441 -train accuracy: 0.636248
epoch: 576 - cost: 0.683253 -MSE: 7.81024014745 -train accuracy: 0.632257
epoch: 577 - cost: 0.681658 -MSE: 8.49092792083 -train accuracy: 0.636792
epoch: 578 - cost: 0.681738 -MSE: 7.783380

epoch: 677 - cost: 0.639329 -MSE: 7.45190796607 -train accuracy: 0.659652
epoch: 678 - cost: 0.639508 -MSE: 6.93295387331 -train accuracy: 0.653846
epoch: 679 - cost: 0.63875 -MSE: 7.43879811266 -train accuracy: 0.661103
epoch: 680 - cost: 0.63893 -MSE: 6.92272498231 -train accuracy: 0.654209
epoch: 681 - cost: 0.638175 -MSE: 7.4264957104 -train accuracy: 0.662373
epoch: 682 - cost: 0.638346 -MSE: 6.91272984632 -train accuracy: 0.654028
epoch: 683 - cost: 0.637597 -MSE: 7.41419938108 -train accuracy: 0.663099
epoch: 684 - cost: 0.63776 -MSE: 6.9027962194 -train accuracy: 0.653846
epoch: 685 - cost: 0.63703 -MSE: 7.40178462326 -train accuracy: 0.663462
epoch: 686 - cost: 0.637191 -MSE: 6.89270515758 -train accuracy: 0.654028
epoch: 687 - cost: 0.636483 -MSE: 7.38898181714 -train accuracy: 0.664187
epoch: 688 - cost: 0.636637 -MSE: 6.88199784005 -train accuracy: 0.654028
epoch: 689 - cost: 0.635935 -MSE: 7.37605125194 -train accuracy: 0.66455
epoch: 690 - cost: 0.636087 -MSE: 6.871674924

epoch: 789 - cost: 0.613328 -MSE: 6.93739217289 -train accuracy: 0.681785
epoch: 790 - cost: 0.613345 -MSE: 6.53589897371 -train accuracy: 0.665276
epoch: 791 - cost: 0.612971 -MSE: 6.93038383112 -train accuracy: 0.682511
epoch: 792 - cost: 0.612987 -MSE: 6.5306954656 -train accuracy: 0.665276
epoch: 793 - cost: 0.612613 -MSE: 6.923760137 -train accuracy: 0.683237
epoch: 794 - cost: 0.612625 -MSE: 6.52557283586 -train accuracy: 0.665457
epoch: 795 - cost: 0.612253 -MSE: 6.91715706448 -train accuracy: 0.683237
epoch: 796 - cost: 0.612264 -MSE: 6.52050736836 -train accuracy: 0.66582
epoch: 797 - cost: 0.611896 -MSE: 6.91043579025 -train accuracy: 0.683599
epoch: 798 - cost: 0.611911 -MSE: 6.51575268789 -train accuracy: 0.66582
epoch: 799 - cost: 0.611551 -MSE: 6.90375823015 -train accuracy: 0.684507
epoch: 800 - cost: 0.611565 -MSE: 6.51063896707 -train accuracy: 0.66582
epoch: 801 - cost: 0.611208 -MSE: 6.8971056218 -train accuracy: 0.685051
epoch: 802 - cost: 0.611221 -MSE: 6.505482889

epoch: 901 - cost: 0.597043 -MSE: 6.54256962277 -train accuracy: 0.693941
epoch: 902 - cost: 0.59701 -MSE: 6.21397962372 -train accuracy: 0.677794
epoch: 903 - cost: 0.596808 -MSE: 6.53596964749 -train accuracy: 0.693941
epoch: 904 - cost: 0.596775 -MSE: 6.20841760516 -train accuracy: 0.678338
epoch: 905 - cost: 0.596577 -MSE: 6.52917394632 -train accuracy: 0.694122
epoch: 906 - cost: 0.596544 -MSE: 6.20259010211 -train accuracy: 0.678701
epoch: 907 - cost: 0.596347 -MSE: 6.52253383206 -train accuracy: 0.694122
epoch: 908 - cost: 0.596314 -MSE: 6.19692305559 -train accuracy: 0.679064
epoch: 909 - cost: 0.596118 -MSE: 6.51600338096 -train accuracy: 0.694122
epoch: 910 - cost: 0.59609 -MSE: 6.19176611661 -train accuracy: 0.678882
epoch: 911 - cost: 0.595899 -MSE: 6.50979253392 -train accuracy: 0.694666
epoch: 912 - cost: 0.595868 -MSE: 6.18617491097 -train accuracy: 0.67852
epoch: 913 - cost: 0.59568 -MSE: 6.50323108831 -train accuracy: 0.695029
epoch: 914 - cost: 0.59565 -MSE: 6.1804973

In [28]:
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print('test accuracy:',sess.run(accuracy,feed_dict={x:test_data,y_:test_labels}))

test accuracy: 0.686502


In [29]:
pred_y=sess.run(y,feed_dict={x:test_data})
mse=tf.reduce_mean(tf.square(pred_y-test_labels))
print('mse:%.4f'%sess.run(mse))

mse:6.2407


In [30]:
#from sklearn.metrics import f1_score,accuracy_score
#print(f1_score(test_labels,predictions),accuracy_score(test_labels,predictions))